## Introduction: Battery

The goal here is to introduce the decision variables and constraints for storage.

We'll add a new asset:
- Battery_1, located in the South
- It can discharge to reduce how much fossil South needs
- It can charge by pulling from imports.

But, in a single-hour model, a battery can 'cheat' unless we are careful.
Because if we let it both charge **and** discharge in the same hour, the model will do _nonsense_ arbitrage inside one time-step.

So, we need to pin one of these two:
- Either, assume battery starts charge with some energy and can only discharge this hour,
- OR, assume it's empty and can only charge this hour,
- OR (more common in planning studies) assume state of charge is fixed both before and after the hour, so net = 0

For now, we'll do the 'starts charged, can discharge' case. That's the most intuitive.
It acts _like_ local backup in South and reduces gas/coal

In [ ]:
# New decision variable
# discharge_batt [MW]; in South

# New parameters:
# batt_power: max discharge power in MW
# batt_energy_max: total usable energy in MWh available right now

In [ ]:
# tbc

Contraints:

1. Discharge cannot exceed power rating: \
$ 0 \leq discharge_\text{ battery} \leq battery_\text{ power max} $


2. Discharge cannot exceed available stored energy: \
$ discharge_\text{ battery} \leq battery_\text{energy max} $

(since we’re doing a 1-hour snapshot, MW and MWh lines up 1:1 here)

And then, we modify South's nodal balance.

Right now, we have

$ generation_\text{ south} + inflow_\text{ to south} - outflow_\text{ from south} = demand_\text{ south} $

with storage discharge located in South, we add it as an internal source:

$ generation_\text{ south} + discharge_\text{ battery} + inflow_\text{ to south} - outflow_\text{ from south} = demand_\text{ south} $

> Economically, we are letting the battery replace fossil output in South in that hour

> We also add an optional small operating cost for discharge (to mimic degradation), e.g 5 EUR/MWh. \

That prevents the model from using battery if generation is already free/cheap

In [ ]:
import pulp as pl


def solve_power_system_with_carbon_and_battery(
        capacity,
        demand_region,
        carbon_price=0,
        batt_power_max=100,  # MW, inverter / discharge limit
        batt_energy_max=100,  # MWh available right now
        batt_discharge_cost=5  # EUR/MWh, optional degradation cost
):
    '''
    Least-cost, multi-region dispatch with:
    - generator capacities
    - transmission limits
    - carbon (CO2) pricing
    - ONE (1) battery in south, that CAN DISCHARGE, at this hour 

    Battery is assumed to be pre-charged, so no charging decision variable will be modelled
    '''

    # --- Define Data ---
    regions = ['North', 'Central', 'South']

    plants = [
        "Wind_1",
        "Wind_2",
        "Nuclear_1",
        "Biomass_1",
        "Coal_1",
        "Gas_1",
        "Gas_2"
    ]

    plant_region = {
        "Wind_1": "North",
        "Nuclear_1": "North",
        "Wind_2": "Central",
        "Biomass_1": "Central",
        "Coal_1": "South",
        "Gas_1": "South",
        "Gas_2": "South"
    }

    # Base marginal cost [€/MWh]
    marginal_cost = {
        "Wind_1": 0,
        "Wind_2": 0,
        "Nuclear_1": 10,
        "Biomass_1": 40,
        "Coal_1": 60,
        "Gas_1": 80,
        "Gas_2": 100
    }

    # Emissions factor [tCO2 / MWh]
    emissions = {
        "Wind_1": 0.0,
        "Wind_2": 0.0,
        "Nuclear_1": 0.0,
        "Biomass_1": 0.0,
        "Coal_1": 0.9,
        "Gas_1": 0.4,
        "Gas_2": 0.4
    }

    # Effective marginal cost including CO2
    effective_cost = {
        p: marginal_cost[p] + carbon_price * emissions[p]
        for p in plants
    }

    # Directed transmission arcs
    arcs = [
        ("North", "Central"),
        ("Central", "North"),
        ("Central", "South"),
        ("South", "Central")
    ]

    line_capacity = {
        ("North", "Central"): 200,
        ("Central", "North"): 200,
        ("Central", "South"): 150,
        ("South", "Central"): 150
    }

    # --- Build the model ---
    model = pl.LpProblem(
        "network_dispatch_with_carbon_and_battery", pl.LpMinimize)

    # Generation decision variables [MW]
    gen = {
        p: pl.LpVariable(
            f"Gen_{p}",
            lowBound=0,
            upBound=capacity[p],
            cat="Continuous"
        )
        for p in plants
    }

    # Flow decision variables [MW]
    flow = {
        (src, dst): pl.LpVariable(
            f'Flow_{src}_to_{dst}',
            lowBound=0,
            upBound=line_capacity[(src, dst)],
            cat='Continuous'
        )
        for (src, dst) in arcs
    }

    # Battery discharge variable (South) [MW]
    '''limited by power and available energy'''
    batt_discharge = pl.LpVariable(
        'Batt_Discharge_South',
        lowBound=0,
        upBound=min(batt_power_max, batt_energy_max),  # MW, 1 hour snapshot
        cat='Continuous'
    )

    # --- Objective ---
    model += (
        pl.lpSum(effective_cost[p] * gen[p]
                 for p in plants) + batt_discharge_cost * batt_discharge
    )

    # Regional nodal balance
    for region in regions:
        gen_in_region = pl.lpSum(
            gen[p] for p in plants if plant_region[p] == region
        )

        inflow_region = pl.lpSum(
            flow[(src, dst)]
            for (src, dst) in arcs
            if dst == region
        )

        outflow_region = pl.lpSum(
            flow[(src, dst)]
            for (src, dst) in arcs
            if src == region
        )

        # Add battery discharge in South region
        if region == 'South':
            model += (
                gen_in_region + inflow_region - outflow_region +
                batt_discharge == demand_region[region]
            ), f'Balance_{region}'
        else:
            model += (
                gen_in_region + inflow_region - outflow_region
                == demand_region[region]
            ), f"Balance_{region}"

    # Solve
    model.solve(pl.PULP_CBC_CMD(msg=False))

    # Report
    print(f"Status: {pl.LpStatus[model.status]}")
    system_cost = pl.value(model.objective)
    print(f"Total System Cost (incl. CO2 & battery): €{system_cost:.2f}\n")

    print("Effective marginal cost [€/MWh]:")
    for p in plants:
        print(f"  {p:<10} : {effective_cost[p]:6.2f} €/MWh")

    print(f"\nBattery discharge (South): {batt_discharge.value():.2f} MW "
          f"(limit {min(batt_power_max, batt_energy_max)} MW)")

    print("\nDispatch by plant [MW]:")
    for p in plants:
        print(
            f"  {p:<10} : {gen[p].value():6.2f} / {capacity[p]} MW "
            f"(region {plant_region[p]})"
        )

    print("\nFlows between regions [MW]:")
    for (src, dst) in arcs:
        val = flow[(src, dst)].value()
        if val > 1e-6:
            print(
                f"  {src} → {dst}: {val:.2f} MW "
                f"(limit {line_capacity[(src, dst)]} MW)"
            )

    print("\nRegional supply/demand check:")
    for region in regions:
        gen_val = sum(
            gen[p].value() for p in plants if plant_region[p] == region
        )
        inflow_val = sum(
            flow[(src, dst)].value()
            for (src, dst) in arcs
            if dst == region
        )
        outflow_val = sum(
            flow[(src, dst)].value()
            for (src, dst) in arcs
            if src == region
        )
        # add batt for south in the print too
        extra = batt_discharge.value() if region == "South" else 0.0
        lhs = gen_val + inflow_val - outflow_val + extra
        print(
            f"  {region:<7} demand {demand_region[region]:6.2f}  |  "
            f"LHS {lhs:6.2f}  "
            f"(gen {gen_val:.2f} + inflow {inflow_val:.2f} "
            f"- outflow {outflow_val:.2f} + batt {extra:.2f})"
        )

    return model

In [4]:
demand_stress = {
    "North":   300,
    "Central": 250,
    "South":   300
}

cap_upgrade = {
    "Wind_1": 100,
    "Wind_2": 100,
    "Nuclear_1": 300,   # upgraded north
    "Biomass_1": 150,
    "Coal_1": 250,
    "Gas_1": 150,
    "Gas_2": 150
}

print("=== CO2 50, with battery in South ===")
solve_power_system_with_carbon_and_battery(
    capacity=cap_upgrade,
    demand_region=demand_stress,
    carbon_price=50,
    batt_power_max=80,      # try 80 MW battery
    batt_energy_max=80,     # assume 80 MWh available
    batt_discharge_cost=5   # small cost
)

=== CO2 50, with battery in South ===
Status: Optimal
Total System Cost (incl. CO2 & battery): €21400.00

Effective marginal cost [€/MWh]:
  Wind_1     :   0.00 €/MWh
  Wind_2     :   0.00 €/MWh
  Nuclear_1  :  10.00 €/MWh
  Biomass_1  :  40.00 €/MWh
  Coal_1     : 105.00 €/MWh
  Gas_1      : 100.00 €/MWh
  Gas_2      : 120.00 €/MWh

Battery discharge (South): 80.00 MW (limit 80 MW)

Dispatch by plant [MW]:
  Wind_1     : 100.00 / 100 MW (region North)
  Wind_2     : 100.00 / 100 MW (region Central)
  Nuclear_1  : 300.00 / 300 MW (region North)
  Biomass_1  : 150.00 / 150 MW (region Central)
  Coal_1     :   0.00 / 250 MW (region South)
  Gas_1      : 120.00 / 150 MW (region South)
  Gas_2      :   0.00 / 150 MW (region South)

Flows between regions [MW]:
  North → Central: 100.00 MW (limit 200 MW)
  Central → South: 100.00 MW (limit 150 MW)

Regional supply/demand check:
  North   demand 300.00  |  LHS 300.00  (gen 400.00 + inflow 0.00 - outflow 100.00 + batt 0.00)
  Central demand 25

network_dispatch_with_carbon_and_battery:
MINIMIZE
5*Batt_Discharge_South + 40.0*Gen_Biomass_1 + 105.0*Gen_Coal_1 + 100.0*Gen_Gas_1 + 120.0*Gen_Gas_2 + 10.0*Gen_Nuclear_1 + 0.0
SUBJECT TO
Balance_North: Flow_Central_to_North - Flow_North_to_Central + Gen_Nuclear_1
 + Gen_Wind_1 = 300

Balance_Central: - Flow_Central_to_North - Flow_Central_to_South
 + Flow_North_to_Central + Flow_South_to_Central + Gen_Biomass_1 + Gen_Wind_2
 = 250

Balance_South: Batt_Discharge_South + Flow_Central_to_South
 - Flow_South_to_Central + Gen_Coal_1 + Gen_Gas_1 + Gen_Gas_2 = 300

VARIABLES
Batt_Discharge_South <= 80 Continuous
Flow_Central_to_North <= 200 Continuous
Flow_Central_to_South <= 150 Continuous
Flow_North_to_Central <= 200 Continuous
Flow_South_to_Central <= 150 Continuous
Gen_Biomass_1 <= 150 Continuous
Gen_Coal_1 <= 250 Continuous
Gen_Gas_1 <= 150 Continuous
Gen_Gas_2 <= 150 Continuous
Gen_Nuclear_1 <= 300 Continuous
Gen_Wind_1 <= 100 Continuous
Gen_Wind_2 <= 100 Continuous